In [1]:
%load_ext autoreload
%autoreload 2
from expressiveness_benchmark.types import Plan, Task, Language, SourceRange, Program
from code_widget.example import CodeWidget
from dataclasses import replace
import json
import pandas as pd

In [2]:
# CHANGE ME!
TASK_ID = 'changing_list'
AUTHOR = 'g'


In [14]:
task = Task(
    id=TASK_ID,
    description="First element you can remove that would not change the mean of the list by more than N",
    plan=[
    ],
    sample_input={
       "list":[{"value":1},{"value":2},{"value":3},{"value":1}],
    },
    sample_output=1,
)
task.save()

prototype = Program(
    task=TASK_ID,
    author=AUTHOR,
    language=''    
)

In [16]:
imp = replace(prototype,
    language='python-imperative',
    source='''
import copy
def get_mean(ls):
    sum = 0
    for e in ls:
        sum += e['value']
    return sum/len(ls)

def changing_list(list):
    start_mean = get_mean(list)
    for i,elem in enumerate(list):
        new_ls = copy.deepcopy(list)
        del new_ls[i]
        mean = get_mean(new_ls)
        if abs(mean - start_mean) < 0.1:
            return i
''')
imp.execute(task)
imp.save()

In [20]:
func = replace(prototype,
    language='python-functional',
    source='''
def changing_list(list):
    start_mean = sum([l['value'] for l in list])/len(list)
    diff = lambda m, sm: abs(m - sm)
    
    for i,elem in enumerate(list):
        new_ls = [x['value'] for x in list if x != elem]
        mean = sum(new_ls)/len(new_ls)
        if diff(mean,start_mean) < 0.1:
            return i
''')
func.execute(task)
func.save()

In [33]:
func = replace(prototype,
    language='sql',
    source='''
SELECT m1.value
FROM list as m1 INNER JOIN
  list as m2 ON (m1.value != m2.value)
  WHERE 
''')
func.execute(task)
func.save()

Mismatch between target and actual output.
Target:     0
0  1
Actual:     0
0  1
1  1
2  1
3  1
4  2
5  2
6  2
7  3
8  3
9  3


AssertionError: 